In [38]:
import folium
import mapclassify
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

from graphly.api_client import SparqlClient

In [39]:
geosparql = SparqlClient("https://geo.ld.admin.ch/query")
sparql = SparqlClient("https://ld.admin.ch/query")

In [40]:
# https://s.zazuko.com/bFtMiF
# Quicker: https://s.zazuko.com/cohgxY
query = """
PREFIX schema: <http://schema.org/>
PREFIX admin: <https://schema.ld.admin.ch/>
PREFIX locn: <http://www.w3.org/ns/locn#>

SELECT ?thoroughfare ?companies ?geom
FROM <https://lindas.admin.ch/foj/zefix>
FROM <https://lindas.admin.ch/territorial>
WHERE {
  VALUES ?muni {<https://ld.admin.ch/municipality/261>}
  {
    SELECT ?thoroughfare (COUNT(?sub) AS ?companies)
    WHERE {
        ?sub a admin:ZefixOrganisation ;
          schema:address/locn:thoroughfare ?thoroughfare;
          admin:municipality ?muni.             
    } 
      GROUP BY ?thoroughfare
      ORDER BY DESC (?companies)
  }

  SERVICE <https://geo.ld.admin.ch/query> {
      GRAPH <urn:bgdi:boundaries:municipalities> {
        ?muni_swisstopo schema:about ?muni.
      }
      
      GRAPH <urn:bgdi:location:streets> {
        ?street_id a <http://www.opengis.net/ont/geosparql#Feature>;
           schema:name ?thoroughfare;
           schema:containedInPlace ?muni_swisstopo;
           <http://www.opengis.net/ont/geosparql#hasGeometry>/<http://www.opengis.net/ont/geosparql#asWKT> ?geom.
      }
   }
  
}
"""
df = sparql.send_query(query)
df = df.set_crs(epsg=4326)

In [41]:
N = 6
classifier = mapclassify.NaturalBreaks(y=df.companies, k=N)
df["bucket"] = df[["companies"]].apply(classifier)
labels = mapclassify.classifiers._get_mpl_labels(classifier, fmt="{:.0f}")
labels[0] = labels[0].replace("0,", "1,")

cmap = plt.get_cmap('inferno')
def style_function(feature):
    bucket = df["bucket"].get(int(feature["id"][-5:]), None)
    if bucket == 0 and df["companies"].get(int(feature["id"][-5:]), None) == 0:
        bucket = None
    return {
        "fillOpacity": 0.6,
        "weight": 3,
        "opacity": 1,
        "fillColor": "red",#red" if bucket is None else mcolors.rgb2hex(cmap((bucket+1)/N)),
        "color": "red", #303030" if bucket is None else mcolors.rgb2hex(cmap((bucket+1)/N)),
    }


highlight_function = lambda x: {'fillColor': '#989898', 
                                'color':'#989898', 
                                'fillOpacity': 0.8}

In [42]:
m = folium.Map(location=[47.15, 8.52], zoom_start=13, tiles="CartoDBdark_matter")

for bucket, label in enumerate(labels):

    feature_group = folium.FeatureGroup(name=label).add_to(m)
    folium.features.GeoJson(
        df[df.bucket == bucket],
        style_function=style_function, 
        control=False,
        highlight_function=highlight_function,
        tooltip=folium.features.GeoJsonTooltip(
            fields=['thoroughfare', "companies"],
            aliases=['Street: ', "Registered companies: "],
            style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
        )
    ).add_to(feature_group)


folium.LayerControl(name="bla").add_to(m)
title_html = '''<h3 align="center" style="font-size:16px"><b>{}</b></h3>'''.format("Where are most companies registered?")   
m.get_root().html.add_child(folium.Element(title_html))

m.save("zug2.html")
m